# <span style="color:gold">**Método del Inverso de la Distancia**</span>

***
### **Editado por: Kevin Alexander Gómez**
#### Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)
Este tutorial está basado en el trabajo de [Hugo Solis](https://github.com/hugosoliss/EDAPythonForGeostatician).
***

### **Descripción**

En este tutorial, revisaremos el método de interpolación **IDW (Inverse Distance Weighting)**.

Este Notebook forma parte del proyecto [**Python para Geólogos**](https://github.com/kevinalexandr19/manual-python-geologia), cuya finalidad es la de facilitar el aprendizaje en Python a estudiantes y profesionales en el campo de la Geología.

***

## **¿En qué consiste el método del Inverso de la Distancia?**
Abreviado como **IDW**, esta técnica de interpolación no-geoestadística es usada frecuentemente en la estimación de variables regionalizadas.\
El valor estimado es una suma ponderada de todos los puntos cercanos.\
Cada punto tiene un factor de ponderación que depende directamente del inverso de su distancia al punto de interpolación y a su vez es elevado a un exponente.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial import KDTree

Empezamos cargando la información:
> Usaremos los métodos `dropna` y `reset_index` para remover las filas con valores vacíos.\
> Seleccionaremos las columnas `xlocation`, `ylocation`, `zlocation` y `gold`.\
> Usaremos la variable `sample` para almacenar la información inicial.

In [ ]:
sample = pd.read_csv("files/ISAACS.csv").dropna().reset_index()
sample = sample[["xlocation", "ylocation", "gold"]]

print(f"Total de muestras: {len(sample)}")
sample.head()

Usando esta información, separaremos el DataFrame en 2 arreglos de Numpy:
> Usaremos la variable `sampleXY` para almacenar las coordenadas XY de las muestras.\
> La variable `sampleAu` será usada para almacenar los valores de Au de las muestras.

In [ ]:
sampleXY = sample[["xlocation", "ylocation"]].to_numpy()
sampleAu = sample["gold"].to_numpy()

Con estos arreglos, visualizaremos la distribución espacial de las leyes de Au:

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"aspect": 1})

# Diagrama de dispersión
im = ax.scatter(x=sampleXY[:, 0], y=sampleXY[:, 1], c=sampleAu, edgecolor="black", cmap="jet", s=30)

# Barra de colores
cbar = fig.colorbar(im, ax=ax, ticks=np.linspace(sampleAu.min(), sampleAu.max(), 9))
cbar.ax.set_title("Au (ppm)", fontsize=15, y=1.01)

# Texto
ax.set_xlabel("Este (m)", fontsize=18)
ax.set_ylabel("Norte (m)", fontsize=18)
ax.set_title("Leyes de Au", fontsize=22, y=1.01)

# Grilla
ax.grid()
ax.set_axisbelow(True) # Puntos por encima de la grilla

plt.show()

## **Grilla 2D para la interpolación**

Estableceremos una grilla de puntos para realizar la interpolación 2D:
- La grilla tendrá 26 x 30 = 780 puntos.
- La distancia entre puntos será de 10 metros en ambas direcciones (X e Y).
- El primer punto (ubicado en la esquina inferior izquierda de la grilla) tendrá las coordenadas (5, 5).
- La extensión total de la grilla será de 260 m x 300 m.

In [ ]:
# Extensión de la grilla
nx, ny = 26, 30 

# Distancia entre cada punto
dx, dy = 10, 10 

# Punto inferior izquierdo
xmin, ymin = 5, 5

# Punto superior derecho
xmax = xmin + (nx * dx)
ymax = ymin + (ny * dy)

Almacenaremos los puntos en un arreglo de nombre `gridXY`:

In [ ]:
# Ubicación de todos los puntos en la grilla
xx, yy = np.meshgrid(np.arange(xmin, xmax, dx), np.arange(ymin, ymax, dy), indexing="ij")
gridXY = np.array([xx.ravel(), yy.ravel()])
print(gridXY.shape)

La grilla tiene 2 dimensiones y contiene 26 x 30 = 780 puntos en total.

Usando este arreglo y la figura anterior, visualizaremos la grilla de estimación:

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"aspect": 1})

# Diagrama de dispersión
ax.scatter(gridXY[0], gridXY[1], c="gray", s=30)
im = ax.scatter(x=sampleXY[:, 0], y=sampleXY[:, 1], c=sampleAu, edgecolor="black", cmap="jet", s=30)

# Barra de colores
cbar = fig.colorbar(im, ax=ax, ticks=np.linspace(sampleAu.min(), sampleAu.max(), 9))
cbar.ax.set_title("Au (ppm)", fontsize=15, y=1.01)

# Texto
ax.set_xlabel("Este (m)", fontsize=18)
ax.set_ylabel("Norte (m)", fontsize=18)
ax.set_title("Grilla de interpolación 2D", fontsize=22, y=1.01)

# Grilla
ax.grid()
ax.set_axisbelow(True)

plt.show()

## **Estimación usando el método IDW**
Interpolaremos el valor de Au en cada punto de la grilla usando el método IDW sobre los valores de Au en las muestras.\
Usaremos la función de interpolación `idw2D`, desarrollada para este tutorial:

In [ ]:
def idw2D(sampleXY: np.ndarray, sampleVar: np.ndarray, gridXY: np.ndarray, nsamples: int = 3, power: int = 1) -> np.ndarray:
    """    Interpolación usando el método IDW en 2 dimensiones.\n
    Parámetros
    ----------
    sampleXY : np.ndarray, arreglo con la ubicación de las muestras en 2D.
    sampleVar : np.ndarray, arreglo con los valores de la variable regionalizada.
    gridXY : np.ndarray, arreglo con la ubicación de los puntos que conforman la grilla de interpolación.
    nsamples: int, representa el número de muestras cercanas utilizadas en la interpolación.
    power: int, representa el exponente que será usado para calcular los factores de ponderación.\n
    """ 
    assert nsamples >= 2, "nsamples debe ser mayor a 2"
    assert power >= 0, "power debe ser mayor o igual a 0"
    
    gridVar = []
    
    for point in zip(gridXY[0], gridXY[1]):
        # Ubicación de n puntos cercanos
        kdt = KDTree(sampleXY)
        d, k = kdt.query(point, k=nsamples)
        nearest = kdt.data[k]
        
        # Interpolación IDW
        if point not in nearest:
            wts = 1 / np.power(d, power)
            wts /= wts.sum(axis=0)
            var = sampleVar[k]
            value = np.sum(var * wts)
            gridVar.append(value)
        
        # Si el punto coincide con la ubicación de una muestra
        else: 
            value = sampleVar[k[0]]
            gridVar.append(value)
    
    gridVar = np.array(gridVar)
    
    return gridVar

In [ ]:
print(idw2D.__doc__)

In [ ]:
gridAu = idw2D(sampleXY=sampleXY, sampleVar=sampleAu, gridXY=gridXY, nsamples=3, power=1)
gridAu.shape

La variable `gridAu` contiene los 780 puntos de la grilla, interpolados con los valores de Au en la muestras.\
Ahora, visualizaremos estos valores interpolados:

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"aspect": 1})

# Diagrama de dispersión
im = ax.scatter(gridXY[0], gridXY[1], c=gridAu, edgecolor="black", cmap="jet", marker="s", s=120)
ax.scatter(sampleXY[:, 0], sampleXY[:, 1], c=sampleAu, edgecolor="black", cmap="jet", s=25)

# Barra de colores
cbar = fig.colorbar(im, ax=ax, ticks=np.linspace(gridAu.min(), gridAu.max(), 9))
cbar.ax.set_title("Au (ppm)", fontsize=15, y=1.01)

# Texto
ax.set_xlabel("Este (m)", fontsize=18)
ax.set_ylabel("Norte (m)", fontsize=18)
ax.set_title("Interpolación de leyes de Au", fontsize=22, y=1.01)

# Grilla
ax.grid()
ax.set_axisbelow(True)

plt.show()

Como podemos observar, el método IDW nos permite estimar los valores de Au en puntos que no han sido muestreados.\
Por último, podemos realizar una visualización interactiva y variar los parámetros `nsamples` y `power` de la función `idw2D`:

In [ ]:
from ipywidgets import interact

In [ ]:
def plot_idw2D(nsamples, power):
    gridAu = idw2D(sampleXY, sampleAu, gridXY, nsamples, power)
    
    # Figura principal
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"aspect": 1})

    # Diagrama de dispersión
    im = ax.scatter(gridXY[0], gridXY[1], c=gridAu, edgecolor="black", cmap="jet", marker="s", s=120)
    ax.scatter(sampleXY[:, 0], sampleXY[:, 1], c=sampleAu, edgecolor="black", cmap="jet", s=25)
    
    # Barra de colores
    cbar = fig.colorbar(im, ax=ax, ticks=np.linspace(gridAu.min(), gridAu.max(), 9))
    cbar.ax.set_title("Au (ppm)", fontsize=15, y=1.01)

    # Texto
    ax.set_xlabel("Este (m)", fontsize=18)
    ax.set_ylabel("Norte (m)", fontsize=18)
    ax.set_title("Interpolación de leyes de Au", fontsize=22, y=1.01)

    # Grilla
    ax.grid()
    ax.set_axisbelow(True)

    plt.show()

interact(plot_idw2D, nsamples=(2, 10, 1), power=(0, 3, 1));

***